<a href="https://colab.research.google.com/github/sayem-eee-kuet/ECE584/blob/main/Single_Object_Motion_Prediction_With_KalmanFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#----------------------------
#--------- Imports-----------
#----------------------------
import cv2 as cv
import numpy as np
import sys
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

MAX_OBJECTS_TO_TRACK = 10

In [4]:
#-------------------------------------------------------#
# Implements Ball motion prediction using Kalman Filter #
#-------------------------------------------------------#

import cv2 as cv
import numpy as np
import sys

# Instantiate OCV kalman filter

class KalmanFilter:

    kf = cv.KalmanFilter(4, 2)
    kf.measurementMatrix = np.array([[1, 0, 0, 0], 
                                     [0, 1, 0, 0]], np.float32)
    dt = 1.0
    # kf.transitionMatrix = np.array([[1, dt, 0, 0], 
    #                                 [0, 1,  0, 0], 
    #                                 [0, 0, 1, dt], 
    #                                 [0, 0, 0, 1]], np.float32)
    

    
    kf.transitionMatrix = np.array([[1, 0, dt, 0], 
                                    [0, 1, 0, dt], 
                                    [0, 0, 1, 0], 
                                    [0, 0, 0, 1]], np.float32)

    def Estimate(self, coordX, coordY):
        ''' This function estimates the position of the object'''
        measured = np.array([[np.float32(coordX)], [np.float32(coordY)]])
        self.kf.correct(measured)
        predicted = self.kf.predict()
        return predicted



#Performs required image processing to get ball coordinated in the video
class ProcessImage:

    def DetectObject(self):

        vid = cv.VideoCapture('/content/gdrive/MyDrive/ECE584/Data/balls.mp4')

        if(vid.isOpened() == False):
            print('Cannot open input video')
            return

        width = int(vid.get(3))
        height = int(vid.get(4))

        # rc, frame = vid.read()

        # print(frame)

        # # Create Kalman Filter Object
        kfObj = KalmanFilter()
        predictedCoords = np.zeros((2, 1), np.float32)

        fr = []

        while(vid.isOpened()):
            rc, frame = vid.read()

            if(rc == True):
                [ballX, ballY] = self.DetectBall(frame)
                predictedCoords = kfObj.Estimate(ballX, ballY)

                # Draw Actual coords from segmentation
                cv.circle(frame, (int(ballX), int(ballY)), 20, [0, 0, 255], 2, 8)
                cv.line(frame,(int(ballX), int(ballY + 20)), 
                        (int(ballX + 50), int(ballY + 20)), [0, 0, 255], 2, 8)
                cv.putText(frame, "Actual", (int(ballX + 50), int(ballY + 20)), 
                           cv.FONT_HERSHEY_SIMPLEX,0.5, [0, 0, 255])  #[50, 200, 250]

                # Draw Kalman Filter Predicted output
                cv.circle(frame, (predictedCoords[0], predictedCoords[1]), 20, [255, 0, 0], 2, 8)
                cv.line(frame, (predictedCoords[0] + 16, predictedCoords[1] - 15), 
                        (predictedCoords[0] + 50, predictedCoords[1] - 30), [255, 0, 0], 2, 8)
                cv.putText(frame, "Predicted", (int(predictedCoords[0] + 50), 
                                                int(predictedCoords[1] - 30)), 
                                                cv.FONT_HERSHEY_SIMPLEX, 0.5, [255, 0, 0]) #  [50, 200, 250]
                # cv2_imshow(frame)
                # im = cv2_imshow(frame)
                fr.append(frame)
                # # plt.imshow(frame, animated=True)

                if (cv.waitKey(300) & 0xFF == ord('q')):
                    break

            else:
                break

        vid.release()
        cv.destroyAllWindows()

        return fr



    # Segment the green ball in a given frame
    def DetectBall(self, frame):

        # Set threshold to filter only green color & Filter it
        lowerBound = np.array([130, 30, 0], dtype = "uint8")
        upperBound = np.array([255, 255, 90], dtype = "uint8")
        greenMask = cv.inRange(frame, lowerBound, upperBound)

        # Dilate
        kernel = np.ones((5, 5), np.uint8)
        greenMaskDilated = cv.dilate(greenMask, kernel)
        #cv.imshow('Thresholded', greenMaskDilated)

        # Find ball blob as it is the biggest green object in the frame
        [nLabels, labels, stats, centroids] = cv.connectedComponentsWithStats(greenMaskDilated, 8, cv.CV_32S)

        # First biggest contour is image border always, Remove it
        stats = np.delete(stats, (0), axis = 0)
        try:
            maxBlobIdx_i, maxBlobIdx_j = np.unravel_index(stats.argmax(), stats.shape)

        # This is our ball coords that needs to be tracked
            ballX = stats[maxBlobIdx_i, 0] + (stats[maxBlobIdx_i, 2]/2)
            ballY = stats[maxBlobIdx_i, 1] + (stats[maxBlobIdx_i, 3]/2)
            return [ballX, ballY]
        except:
               pass

        return [0,0]

In [5]:
# Main Function
def main():

    processImg = ProcessImage()
    f = processImg.DetectObject()
    # print(type(f))
    f = np.asanyarray(f)
    # print(f.shape)
    lenght, height, width, layer = f.shape
    FPS = 10
    time = 0.07
    fourcc = cv.VideoWriter_fourcc(*'MP42')
    video = cv.VideoWriter('/content/gdrive/MyDrive/ECE584/Output/single.avi', fourcc, float(FPS), (width, height))

    for i in range(len(f)):
      video.write(f[i])
    
    video.release()
    


if __name__ == "__main__":
    main()

print('Program Completed! See output folder...')

Program Completed! See output folder...


In [ ]:


# from cv2 import VideoWriter, VideoWriter_fourcc

# width = 1280
# height = 720
# FPS = 10
# seconds = 0.07

# fourcc = VideoWriter_fourcc(*'MP42')
# video = VideoWriter('/content/gdrive/MyDrive/ECE584/Output/output.avi', fourcc, float(FPS), (width, height))

# for i in range(len(f)):
#     video.write(f[i])
# video.release()



In [ ]:
# print(len(f))
# for i in range(len(f)):
#   im = cv2_imshow(f[i])

In [ ]:
# processImg = ProcessImage()
# processImg.DetectObject()

In [ ]:
# f10 = lambda x, t : 1 / (1 + 16 * (x - t)**2)
# xs10 = np.linspace(-1, 1, 101)

# fig, ax = plt.subplots()

# ax.set_xlim(( -1, 1))
# ax.set_ylim((0, 1.5))

# line, = ax.plot([], [], lw=2)

# def init():
#     line.set_data([], [])
#     return (line,)

# def animate(t):
#     line.set_data(xs10, f10(xs10, t))
#     return (line,)
# anim = animation.FuncAnimation(fig, animate, init_func=init,
#                                frames=np.arange(0, 1, 0.01), blit=True)

# HTML(anim.to_html5_video())